The solution is based on the langchain module, which implemented the RAG agent approach. The approach uses langgraph to graphically transition between different program functionalities. The PaS hinting technique involves planning a solution to a problem and decomposing the problem into smaller sub-problems to help the LLM find a solution. In the context of the RAG technique, this allows the LLM to better “understand” the question, extract the most important elements of the question, and ultimately produce a better quality answer.

In [8]:
%%capture
%pip install -U --quiet langchain-community tiktoken langchainhub langchain langgraph langchain-text-splitters langchain_ollama dotenv

In [20]:
%%capture
!pip install langchain_ollama
!pip install langchain_benchmarks

Imports

In [28]:
import getpass
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from typing import Annotated, Sequence, TypedDict, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langgraph.prebuilt import tools_condition

In [51]:
from dotenv import load_dotenv
load_dotenv('.env')

True

# Graphstate

In [52]:
class GraphtState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Docs

In [53]:
from langchain_benchmarks import clone_public_dataset, registry

registry = registry.filter(Type="RetrievalTask")
langchain_docs = registry["LangChain Docs Q&A"]
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)
docs = list(langchain_docs.get_docs())
print(repr(docs[0])[:100] + "...")

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)
embeddings = OllamaEmbeddings(model="llama3")


Dataset LangChain Docs Q&A already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/7cc86f41-37ac-54c3-9fc1-217a4842cc96/datasets/e1e24afe-7ff4-4786-9d87-a975e1a568c4.
Document(metadata={'changefreq': 'weekly', 'description': 'Example code for building applications wi...


# Vectorstore and Retriever

In [ ]:
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
)

retriever = faiss_vectorstore.as_retriever()

In [ ]:
retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_langchain_doc",
    "Return information about a particular topic from langchain docs",
)

tools = [retriever_tool]

# Graph's edges

In [ ]:
model_name = "llama3.1"

### Edges

def grade_documents(state) -> Literal["generate", "rewrite"]:
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """

    print("---CHECK RELEVANCE---")

    # Data model
    class grade(BaseModel):
        """Binary score for relevance check."""

        binary_score: str = Field(description="Relevance score '1' or '0'")

    # LLM
    model = ChatOllama(temperature=0, model=model_name, streaming=True)

    # LLM with tool and validation
    llm_with_tool = model.with_structured_output(grade)

    # Prompt
    prompt = PromptTemplate(
        template="""You are a grader assessing relevance of a retrieved document to a user question. \n
        Here is the retrieved document: \n\n {context} \n\n
        Here is the user question: {question} \n
        If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
        Give a binary score '1' for 'yes' or '0' for 'no' score to indicate whether the document is relevant to the question.""",
        input_variables=["context", "question"],
    )

    # Chain
    chain = prompt | llm_with_tool

    messages = state["messages"]
    last_message = messages[-1]

    question = messages[0].content
    docs = last_message.content

    scored_result = chain.invoke({"question": question, "context": docs})

    score = scored_result.binary_score

    if score == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "generate"

    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(score)
        return "rewrite"


### Nodes

def agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    model = ChatOllama(temperature=0, streaming=True, model=model_name)
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


def rewrite(state):
    """
    Transform the query to produce a better question.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with re-phrased question
    """

    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    question = messages[0].content

    msg = [
        HumanMessage(
            content=f""" \n
            Examine the input and try to determine the underlying semantic intent or meaning.
            Here is the initial question:
            \n ------- \n
            {question}
            \n ------- \n
            Formulate an improved question: """,
        )
    ]

    # Grader
    model = ChatOllama(temperature=0, 
                       model=model_name, 
                       streaming=True)
    
    response = model.invoke(msg)
    return {"messages": [response]}


def generate(state):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    messages = state["messages"]
    question = messages[0].content
    last_message = messages[-1]

    docs = last_message.content

    # Prompt
    prompt = hub.pull("rlm/rag-prompt")

    # LLM
    llm = ChatOllama(model_name=model_name, 
                     temperature=0, 
                     streaming=True)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}


print("*" * 20 + "Prompt[rlm/rag-prompt]" + "*" * 20)
prompt = hub.pull("rlm/rag-prompt").pretty_print()  # Show what the prompt looks like

Graph

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# Define a new graph
workflow = StateGraph(GraphtState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # Re-writing the question
workflow.add_node(
    "generate", generate
)  # Generating a response after we know the documents are relevant
# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile()

Graph image

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

Evaluation

In [ ]:
import pprint


query = "What is the fine tuning process for transformers? What are the best practices?"

pas_prompt = f"""question: {query}
First, let's understand the question and develop a plan to answer it.
Then let's implement the plan and solve the problem step by step.
step by step
"""

inputs = {
    "messages": [
        ("user", pas_prompt),
    ]
}
for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")